In [3]:
# Import packages and initiate engine
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
# Open the connection
with engine.connect() as con:
    Bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
    Facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)
    Members = pd.read_sql_query("SELECT * FROM Members", engine)

In [5]:
Bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [6]:
Facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
Members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [8]:
# Answer Case Study PART 2 Questions
# Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue.
Bookings_Facilities = Bookings.merge(Facilities, on = 'facid', how = 'left')
Bookings_Facilities.head()

,bookid,facid,memid,starttime,slots,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,3,1,2012-07-03 11:00:00,2,Table Tennis,0.0,5.0,320,10
1,1,4,1,2012-07-03 08:00:00,2,Massage Room 1,9.9,80.0,4000,3000
2,2,6,0,2012-07-03 18:00:00,2,Squash Court,3.5,17.5,5000,80
3,3,7,1,2012-07-03 19:00:00,2,Snooker Table,0.0,5.0,450,15
4,4,8,1,2012-07-03 10:00:00,1,Pool Table,0.0,5.0,400,15


In [12]:
# Calculate revenue for all bookings
Bookings_Facilities['revenue'] = 0
guestbooking = Bookings_Facilities['memid'] == 0
memberbooking = Bookings_Facilities['memid'] != 0
Bookings_Facilities.loc[guestbooking, 'revenue'] = Bookings_Facilities['guestcost']*Bookings_Facilities['slots']
Bookings_Facilities.loc[memberbooking, 'revenue'] = Bookings_Facilities['membercost']*Bookings_Facilities['slots']

In [18]:
# Group the dataframe by facility name
result = Bookings_Facilities.groupby(['facid','name'])['revenue'].sum().reset_index()
result = result[result['revenue'] < 1000 ]

In [19]:
result.sort_values('revenue')
print(result)

   facid           name  revenue
3      3   Table Tennis    180.0
7      7  Snooker Table    240.0
8      8     Pool Table    270.0


In [51]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
# Filter members only
members_only = Members.loc[Members['memid'] != 0, ['memid', 'surname','firstname','recommendedby']]
members_only['recommendedby'] = pd.to_numeric(members_only['recommendedby'])

# Fiter members who are recommended only
recommended_only = members_only.dropna(subset = ['recommendedby'])
recommended_only['recommendedby'] = recommended_only['recommendedby'].astype(int)

/tmp/ipykernel_51980/2461527524.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_only['recommendedby'] = recommended_only['recommendedby'].astype(int)


In [52]:
recommended_only.head()

,memid,surname,firstname,recommendedby
4,4,Joplette,Janice,1
5,5,Butters,Gerald,1
7,7,Dare,Nancy,4
8,8,Boothe,Tim,3
9,9,Stibbons,Ponder,6


In [59]:
# Merge with members table to display name of members who recommended others
result = recommended_only.merge(members_only, how = 'right', left_on = 'recommendedby', right_on = 'memid', suffixes=('_recc','_exist'))
result = result.dropna()

In [65]:
final_result = result[['memid_recc','surname_recc','firstname_recc','recommendedby_recc','surname_exist','firstname_exist']].sort_values(['surname_recc','firstname_recc'])
print(final_result)

    memid_recc surname_recc firstname_recc  recommendedby_recc surname_exist  \
16        15.0        Bader       Florence                 9.0      Stibbons   
15        12.0        Baker           Anne                 9.0      Stibbons   
24        22.0       Coplin           Joan                16.0         Baker   
9          7.0         Dare          Nancy                 4.0      Joplette   
11        20.0      Genting        Matthew                 5.0       Butters   
34        35.0         Hunt           John                30.0       Purview   
10        11.0        Jones          David                 4.0      Joplette   
18        26.0        Jones        Douglas                11.0         Jones   
26        27.0       Rumney      Henrietta                20.0       Genting   
23        24.0       Sarwin      Ramnaresh                15.0         Bader   

   firstname_exist  
16          Ponder  
15          Ponder  
24         Timothy  
9           Janice  
11          Ge

In [80]:
# Q12: Find the facilities with their usage by member, but not guests
member_fac_usage = pd.read_sql_query(
    "SELECT f.facid, f.name, COUNT(memid) AS bookingcount, SUM(slots) AS total_slots FROM Bookings b INNER JOIN Facilities f ON f.facid = b.facid WHERE memid != 0 GROUP BY f.name "
, engine)

In [81]:
print(member_fac_usage)

   facid             name  bookingcount  total_slots
0      2  Badminton Court           344         1086
1      4   Massage Room 1           421          884
2      5   Massage Room 2            27           54
3      8       Pool Table           783          856
4      7    Snooker Table           421          860
5      6     Squash Court           195          418
6      3     Table Tennis           385          794
7      0   Tennis Court 1           308          957
8      1   Tennis Court 2           276          882


In [78]:
# Q13: Find the facilities usage by month, but not guests
month_fac_usage = pd.read_sql_query(
    "SELECT STRFTIME('%m', starttime) AS month, COUNT(bookid) AS bookingcount, SUM(slots) AS total_slots FROM Bookings WHERE memid != 0 GROUP BY month ORDER BY month", engine
)

In [79]:
print(month_fac_usage)

  month  bookingcount  total_slots
0    07           480         1061
1    08          1168         2531
2    09          1512         3199
